In [ ]:
# Add description here
#

In [ ]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [1]:
# If this task has dependencies, declare them in the YAML spec and leave this
# as None
upstream = None

# This is a placeholder, leave it as None
product = None

In [ ]:
# Parameters
study1a1b_csv = "D:\\unibas\\new_embedding_risk_perception\\reanalysis\\data\\raw\\bhatia\\Study 1A and 1B - Psychometric Dimensions.csv"
study2_csv = "D:\\unibas\\new_embedding_risk_perception\\reanalysis\\data\\raw\\bhatia\\Study 2 - Psychometric Dimensions.csv"
product = {
    "nb": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\process_psych.ipynb",
    "data": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\psych_mean.csv",
}


In [10]:
import os
import pandas as pd

from scipy.stats import spearmanr
from sklearn.model_selection import ShuffleSplit

In [39]:
def load_bhatia_rawdata(filename:str, fieldpos:int):
    df_std1 = pd.read_csv(filename, header=1).iloc[1:]
    meta_cols = list(df_std1.columns[:18]) + list(df_std1.columns[-2:])
    risk_cols = [x for x in df_std1.columns if x not in meta_cols]
    df_risk = df_std1[risk_cols]
    
    new_cols = []
    counter =0
    prv_name = None
    for x in df_risk.columns:
        x = x.split(' - ')[fieldpos].strip()
        if x == 'waste inceneration':
            x = 'waste incineration'
        counter +=1
        if prv_name is None or prv_name != x:
            counter = 1
            prv_name = x       
        new_cols.append(f'{x}_{counter}')
    
    
    df_risk.columns = new_cols # [f'{x}_{i}' for i, x in enumerate(new_cols)]
    
    return df_risk

In [40]:
dic_filename = {
    'study1a1b': {'filename':study1a1b_csv, 'fieldpos':0}, 
    'study2': {'filename':study2_csv, 'fieldpos':1}
}
dt_raws = {key: load_bhatia_rawdata(**augments) for key, augments in dic_filename.items()}

In [41]:
df_raw = pd.concat(dt_raws.values()).astype(float)

In [42]:
def spearman_brown_coefficient(half_1, half_2):
    rho, _ = spearmanr(half_1, half_2)
    rho_sb = (2*rho)/(1+rho)
    return rho_sb

def calc_mean(ts1, rand_split):
    ts1 = ts1.dropna().reset_index(drop=True)
    idx1, idx2 = next(rand_split.split(ts1))
    return ts1[idx1].mean(), ts1[idx2].mean()
num_fold = 10

ts_rho_sb = []
for i in range(num_fold):
    rand_split = ShuffleSplit(n_splits=1, test_size=0.5, random_state=i)
    df_raw_split_mean = df_raw.apply(lambda ts1: calc_mean(ts1, rand_split))
    rho_sb = spearman_brown_coefficient(df_raw_split_mean.loc[0,:], df_raw_split_mean.loc[1,:])
    ts_rho_sb.append(rho_sb)

ts_rho_sb = pd.Series(ts_rho_sb)

# print mean and std of Spearman-Brown reliability coefficient
print('Mean of Spearman-Brown reliability coefficient: ', ts_rho_sb.mean())
print('Std of Spearman-Brown reliability coefficient: ', ts_rho_sb.std())

Mean of Spearman-Brown reliability coefficient:  0.8655443606263153
Std of Spearman-Brown reliability coefficient:  0.003858146677937164


In [87]:
df_raw_stack = df_raw.mean(axis=0).reset_index()
df_raw_stack.columns = ['word','scale']


In [89]:
df_raw_stack['level'] = df_raw_stack['word'].apply(lambda x : x.split('_')[1])
df_raw_stack['word'] = df_raw_stack['word'].apply(lambda x : x.split('_')[0])

In [92]:
df_raw_pivot = df_raw_stack.pivot(index='word', columns='level', values='scale')

In [97]:
df_raw_pivot.columns = ['voluntary_involuntary', 'notFatal_fatal', 'immediate_delayed',
                      'calm_dread', 'chronic_catastrophic', 'controllable_uncontrollable',
                      'known_unknownSci', 'known_unknown', 'new_old']
df_raw_pivot = df_raw_pivot.reset_index(names='source')

In [98]:
df_raw_pivot

,source,voluntary_involuntary,notFatal_fatal,immediate_delayed,calm_dread,chronic_catastrophic,controllable_uncontrollable,known_unknownSci,known_unknown,new_old
0,ISIS,6.200000,2.771429,3.142857,3.542857,5.714286,2.342857,6.114286,5.914286,5.942857
1,Putin,6.038462,5.076923,4.423077,4.538462,4.807692,3.692308,4.807692,4.384615,4.000000
2,Trump,4.941176,5.235294,3.617647,3.764706,5.294118,2.147059,4.470588,4.558824,3.617647
3,accountant,2.083333,6.250000,4.083333,4.333333,3.250000,5.166667,1.333333,1.250000,2.000000
4,acetaminophen,1.444444,5.888889,2.777778,1.555556,2.222222,5.333333,1.555556,2.666667,3.666667
...,...,...,...,...,...,...,...,...,...,...
424,writing,2.227273,5.863636,3.659091,3.454545,2.545455,5.545455,2.272727,1.863636,1.636364
425,xanax,2.083333,5.250000,2.750000,2.083333,2.416667,3.666667,1.666667,2.666667,3.416667
426,xray,3.000000,5.714286,2.571429,1.500000,2.928571,5.428571,2.000000,2.785714,3.642857
427,yardwork,1.826087,4.217391,3.391304,3.478261,2.608696,4.565217,2.217391,1.739130,2.130435


In [99]:
df_raw_pivot.to_csv(product['data'], index=False)